### Query SRTM dataset metrics for select protected area vectors (WDPA and DAS)
#### From JavaScript here: https://code.earthengine.google.com/5806d067c03a964285f573ad61f07639

Import libraries

In [1]:
import ee, pandas as pd, numpy as np #geopandas,shapely not in current environment - docker container
import seaborn as sns
from IPython import display ##used to display images
from matplotlib import pyplot
from IPython.display import Image

Initialize Earth Engine

In [2]:
ee.Initialize()

Import FeatureCollection

In [3]:
# import das_sites - personal EE asset
das_sites = ee.FeatureCollection('users/jmasselink/das_sites')
wdpa = ee.FeatureCollection('')

In [5]:
## perhaps will help handle the feature collection geometry
# not sure what else this is supposed to accomplish or when it is supposed to be called.
# def feature_das(input_value): 
#     name = ee.Str(input_value)
#     das_sites = ee.FeatureCollection('users/jmasselink/das_sites', 'geometry') ##//ft:1NJIeeE0BzZLNtqA3-WSyxzGg2jxg59K4zD249QCv')
#     das_sites = das_sites.filter(ee.Filter.eq('name', name))
#     das_sites_name = ee.String(ee.Feature(toList(1).get(0)).get('name'))

In [6]:
## shows options and parameters on Computed Object
# help(das_sites)

Count the number of unique features in FeatureCollection using aggregate_count()

In [7]:
das_sites_count = das_sites.aggregate_count('name')
das_sites_count.getInfo()

78

Create feature class to hold DAS features

In [8]:
def calc_elevation_stats(feature):  #, resolution = 100
#     for f in fc:
    print(feature)
    result = dict()

    # calculate mean elevation across feature
    mean = srtm.reduceRegion(ee.Reducer.mean(), feature, 100)
    result['mean'] = mean.getInfo()
    
    # calculate min elevation across feature
#     Min = srtm.reduceRegion(ee.Reducer.min(), feature, 100)
#     result['Min'] = Min.getInfo()
    
    # calculate max elevation across feature
#     Max = srtm.reduceRegion(ee.Reducer.min(), feature, 100)
#     result['Max'] = Max.getInfo()
   
    return result

In [9]:
# function to pass feature from FC to 
def foobar(feature):
    return feature.get('geometry') #.set({areaHa: feature.geometry().area().divide(100 * 100)})

fc = das_sites.map(foobar, opt_dropNulls=None)

In [10]:
help(fc)

Help on FeatureCollection in module ee.featurecollection object:

class FeatureCollection(ee.collection.Collection)
 |  A representation of a FeatureCollection.
 |  
 |  Method resolution order:
 |      FeatureCollection
 |      ee.collection.Collection
 |      ee.element.Element
 |      ee.computedobject.ComputedObject
 |      ee.encodable.Encodable
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, args, opt_column=None)
 |      Constructs a collection features.
 |      
 |      Args:
 |        args: constructor argument.  One of:
 |            1) A string - assumed to be the name of a collection.
 |            2) A number - assumed to be the ID of a Fusion Table.
 |            3) A geometry.
 |            4) A feature.
 |            5) An array of features.
 |            6) A computed object - reinterpreted as a collection.
 |        opt_column: The name of the geometry column to use. Only useful with the
 |            string or number constructor argu

In [11]:
# das_sites.map(calc_elevation_stats)
# for f in fc.iterate():
#     print(f)

In [42]:
## worked in from here: https://groups.google.com/d/msg/google-earth-engine-developers/ENqC9cRJnPU/TS9xYpFkC4wJ
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
   
    # Features is a list of dictionaries with the output
    features = fc.getInfo()['features']
#     features['geometry']
    
    dictArray = []
    
    for f in features:
        # store all attributes in a dict
        attr = f['properties']
#         if attr['name'] != 'Lewa':
#             continue
        
        # Treat geometry separately
        # attr['geometry'] = f['geometry'] #GeoJSON feature
#         def add_area(f):
#             f.set({})
#             f.map()
        
#         f.map()

        if f.get('geometry') : # reads for non-null geometry values
            elevation_stats = calc_elevation_stats(f)
            attr['mean_elevation'] = elevation_stats['mean']['elevation']
#             attr['min_elevation'] = elevation_stats['min']['elevation']
#             attr['max_elevation'] = elevation_stats['max']['elevation']
    
        # attr['geometrytype'] = f['geometry']['type']
        dictArray.append(attr)
        
    # adapted GeoDataFrame (GeoPandas) to DataFrame (Pandas)
    df = pd.DataFrame(dictArray) #
    
    # select attributes to display
    return df

Call fc2df function on das_sites Feature Collection

In [43]:
df = fc2df(das_sites)
# f[['country', 'desig_eng', 'name', 'status']] #, 'mean_elevation']]
df.columns

AttributeError: 'dict' object has no attribute 'getInfo'

In [17]:
das_sites.getInfo()['features']

[{u'geometry': {u'coordinates': [[[-1.7115196582823151, 11.132614702070297],
     [-1.7114298739250717, 11.132453049734506],
     [-1.711245705270137, 11.13223295665976],
     [-1.7109582493882942, 11.131909486636218],
     [-1.71070225646291, 11.131626594965683],
     [-1.7106258298817758, 11.131478338425099],
     [-1.7103698389141295, 11.13120434129667],
     [-1.7100104894994326, 11.130970821082576],
     [-1.709723041786339, 11.130773159717366],
     [-1.7087529012403564, 11.130427301930645],
     [-1.7085148027517474, 11.13031052634559],
     [-1.7082273649306594, 11.130090414814104],
     [-1.7078500778521664, 11.129672735047228],
     [-1.7071358753331416, 11.128936101884488],
     [-1.7065789746782085, 11.128316271346314],
     [-1.7062376113115982, 11.127813172816099],
     [-1.7059231824850902, 11.127368551404201],
     [-1.7056850995003783, 11.126991280155382],
     [-1.705002389493773, 11.125679705007693],
     [-1.7047284033715369, 11.12514075168211],
     [-1.70436012363

In [37]:
# Select only one state at a time
# sceneShape = sceneShape.filter(ee.Filter.eq('state_id', state_id))
# sceneShapeName = ee.String(ee.Feature(sceneShape.toList(1).get(0)).get('name'))
das_features = das_sites.filter(ee.Filter.eq('name', name))
das_features_name = ee.String(ee.Feature(das_features.toList(1).get(0)).get('name'))

AttributeError: 'dict' object has no attribute 'filter'

In [38]:
das_feature = ee.Feature(None)
das_feature = das_feature.set('name', das_sites.name)
das_feature = das_feature.set('geom', ee.Geometry)
das_feature = das_feature.set('area', das_feature.area)
# das_feature = feature.set('area', das_sites)
# das_sites_table = das_sites.read_file('das_sites')
# das_sites_table.head()

AttributeError: 'dict' object has no attribute 'name'

In [39]:
# area = das_feature.area #.getInfo()
print(das_feature.area)

<bound method Feature.Feature.area of <ee.feature.Feature object at 0x7f197d7a9f50>>


Count the number of unique features in FeatureCollection using aggregate_count_distinct()

In [ ]:
das_sites_count_name = das_sites.aggregate_count_distinct('name')
das_sites_count_name.getInfo()

In [ ]:
help(das_sites.aggregate_count_distinct)

Shows ee.ComputedObject() properties

In [25]:
print(das_sites_count)

ee.ComputedObject({
  "type": "Invocation", 
  "arguments": {
    "property": "name", 
    "collection": {
      "type": "Invocation", 
      "arguments": {
        "tableId": "users/jmasselink/das_sites"
      }, 
      "functionName": "Collection.loadTable"
    }
  }, 
  "functionName": "AggregateFeatureCollection.count"
})


In [34]:
# create Pandas data frame to read ee.FeatureCollection
# das_df = pd.Series(das_sites)
das_sites = {}
das_df = pd.DataFrame

In [35]:
das_df.head()

TypeError: unbound method head() must be called with DataFrame instance as first argument (got nothing instead)

**Add WDPA as FeatureCollection**

In [36]:
# import WDPA
# https://code.earthengine.google.com/dataset/WCMC/WDPA/current/polygons
wdpa_polygons = ee.FeatureCollection('WCMC/WDPA/current/polygons') 

Count Distinct Names in WDPA dataset

In [21]:
wdpa_count = wdpa_polygons.aggregate_count_distinct('NAME')

In [22]:
wdpa_count.getInfo()

198888

Count Distinct PA Types in WDPA dataset

In [23]:
wdpa_pa_type_count = wdpa_polygons.aggregate_count_distinct('DESIG_ENG')

In [24]:
wdpa_pa_type_count.getInfo()

1565

Count Terrestrial National Parks in WDPA FeatureCollection

In [ ]:
# double filter of WDPA dataset - Non marine & National Park designation
wdpa_np = wdpa_polygons.filter(ee.Filter().eq('MARINE', '0')).filter(ee.Filter().eq('DESIG_ENG', 'National Park'))
wdpa_np_count = wdpa_np.aggregate_count_distinct('NAME')
wdpa_np_count.getInfo()

#### Accessing Feature Class parameters

Count National Parks in Zambia and list them

In [ ]:
zmb_np = wdpa_polygons.filter(ee.Filter().eq('ISO3', 'ZMB')).filter(ee.Filter().eq('DESIG_ENG', 'National Park'))
zmp_np_count = zmb_np.aggregate_count_distinct('NAME')
zmp_np_count.getInfo()

In [ ]:
fc2df(zmb_np)

In [ ]:
zmp_np_name = pd.Series(zmp_np_count)
zmp_np_name

Get Info on Area Feature Collection

In [ ]:
zmb_nb_histo = zmb_np.aggregate_histogram('GIS_AREA')
area = zmb_nb_histo.getInfo()
# zmb_nb_histo = zmb_np.aggregate_histogram('GIS_AREA')
# area = zmb_nb_histo.getInfo()

In [ ]:
zmb_np_area = []
zmb_np_area = zmb_np.aggregate_histogram('GIS_AREA')

Get a list of names of Zambia national parks

In [ ]:
zmb_np_names = zmb_np_count.toList('NAME') #get('NAME').getInfo()  toDictionary

In [ ]:
print(zmb_np_names)

Write Feature Collection to a Pandas DataFrame

In [ ]:
zmb_np_region = srtm.getRegion(zmb_np, 'EPSG:4326').getInfo()
df = DataFrame(data=zmb_np, columns='NAME')
df.to_csv()
# def buildTable():
# np = zmb_np.reduceColumns(ee.Reducer.toList(),['NAME']).get'list')
# np.getInfo()

In [ ]:
list_sites = das_sites.aggregate_count("name")  ##aggregate_array
print(list_sites)

#### Query for a name

In [ ]:
lewa = (das_sites.filter(ee.Filter().eq('name', 'Lewa')))
lewa_geo = lewa.geometry() #.area()
infoLewa = lewa_geo.getInfo()

Calculate area on feature geometry

In [ ]:
# infoLewa.Feature.area()
areaLewa = infoLewa.area()
areaLewa.getInfo()

## Load asset Feature Collection
Head back to the [Earth Engine Public Data Catalog](https://earthengine.google.org/#index), search for **SRTM**, and load up the data description page for the **SRTM** dataset. The ID for this Image Collection is **CGIAR/SRTM90_V4**.

In [ ]:
srtm = ee.Image('CGIAR/SRTM90_V4')
Image(url=srtm.getThumbUrl())

In [ ]:
Image(url=srtm.getThumbUrl({'min':0, 'max':250}))

Calculate Mean elevation

In [ ]:
elevation_stats(lewa)

In [ ]:
avg_elevation = srtm.reduceRegion(ee.Reducer.mean(), lewa, 100)
avg_elevation.getInfo()

Calculate Min elevation

In [ ]:
min_elevation = srtm.reduceRegion(ee.Reducer.min(), lewa, 100)
min_elevation.getInfo()

Calculate Max elevation

In [ ]:
max_elevation = srtm.reduceRegion(ee.Reducer.max(), lewa, 100)
max_elevation.getInfo()

Calculate area of feature

Calculate slope in area

In [ ]:
slope = ee.Terrain.slope(srtm)
Image(url=slope.getThumbUrl())

Calculate mean slope

In [ ]:
avg_slope = slope.reduceRegion(ee.Reducer.mean(), lewa, 30)
avg_slope.getInfo()

Calculate max slope

In [ ]:
max_slope = slope.reduceRegion(ee.Reducer.max(), lewa, 30)
max_slope.getInfo()

Calculate min slope

In [ ]:
min_slope = slope.reduceRegion(ee.Reducer.min(), lewa, 30)
min_slope.getInfo()

In [ ]:
point = ee.Geometry.Point(37.4256, 0.2067)
buffer_point = point.buffer(40000).bounds().getInfo()['coordinates']
# region_lewa = srtm.filterBounds(lewa).getInfo()#['coordinates']
Image(url=srtm.getThumbUrl({'min':0, 'max':5000, 'region':buffer_point}))

In [ ]:
point = ee.Geometry.Point(37.4256, 0.2067)
buffer_point = point.buffer(40000).bounds().getInfo()['coordinates']
# region_lewa = srtm.filterBounds(lewa).getInfo()#['coordinates']
Image(url=srtm.getThumbUrl({'min':0, 'max':5000, 'region':buffer_point}))

In [ ]:
avg_elevation = ee.Reducer.mean().setOutputs(srtm)
print(avg_elevation)

In [ ]:
hillshade = ee.Terrain.hillshade(srtm) #.visualize(forceRgbOutput=True)

In [ ]:
hillshade.getInfo()

In [ ]:
Image(url=hillshade.getThumbUrl())

In [ ]:
Image(url=hillshade.getThumbUrl({'hillshade': hillshade}))

In [ ]:
# allregion = das_sites.geometry().bounds()
allregion = ee.Geometry.Polygon([-170, 80, 0, 80, 170, 80, 170, -80, 10, -80, -170, -80], None, False)

In [ ]:
def print_image(image,region):
  display.display(display.Image(ee.data.getThumbnail({
      'image': image.serialize(),
      'dimensions': 500,
      'region':lewa
})))

In [ ]:
print_image(srtm, lewa)

In [ ]:
### Create the ele tracks layer
empty1 = ee.Image().byte()
painted=empty1.paint(featureCollection=eleTrack,
  color=1,
  width=5).visualize(palette=['FF0000'],forceRgbOutput=True)

In [ ]:
# ### Create image collection from layers and mosaic in desired draw order
outImg = ee.ImageCollection([hillshade,painted]).mosaic()
# print(outImg.getThumbURL()) #getting the unbounded error here ()
print(outImg.getThumbURL("region": allregion))

In [ ]:
print_image(outImg,allregion) #getting http 400 error here

In [ ]:
# Image(url=srtm.getThumbUrl({'min':0, 'max':1000, 'region':lewa_elevation})) #region_lewa